<a href="https://colab.research.google.com/github/iam-fern/Research/blob/main/sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.random import normal
import statistics

In [13]:
# numpy.random.normal(loc=0.0, scale=1.0, size=None)
# generate sample of 200 values that follow a normal distribution 
# data = normal(loc=0, scale=1, size=200)

data = pd.DataFrame({"zt1" : normal(0, 1, 50000),
                     "zt2" : normal(0, 1, 50000),
                     "zt3" : normal(0, 1, 50000)})
data

,zt1,zt2,zt3
0,1.113729,-0.606795,-0.891996
1,-0.524049,-0.113129,-2.134784
2,0.540244,0.840620,-3.064373
3,-1.082320,-0.527977,1.267484
4,0.505567,0.611488,-0.470201
...,...,...,...
49995,1.108176,-0.379563,0.215550
49996,-1.757373,0.313176,-0.300915
49997,1.229474,-2.235324,-1.923630
49998,-0.722606,0.607819,0.339424


In [14]:
n=50
rep=0
table_data_all=pd.DataFrame({"rep":[]})
data_copy=data.copy()
while not data_copy.empty:
    #=========================
    if data_copy.shape[0]>n:
        index=data_copy.sample(n).index
        data_new=data_copy.loc[index,:]
        data_copy=data_copy.drop(index)
    else:
        data_new=data_copy.copy()
        data_copy=pd.DataFrame({"":[]})
    #=========================
    p = 0

    Zt1 = data_new['zt1']
    Zt2 = data_new['zt2']
    Zt3 = data_new['zt3']

    xt1 = (1-(p**2))**(1/2)*Zt1
    xt2 = (1-(p**2))**(1/2)*Zt2
    xt3 = (1-(p**2))**(1/2)*Zt3

    new_data = pd.DataFrame({'Xt1' : data_new['zt1'],
                            'Xt2' : data_new['zt2'],
                            'Xt3' : data_new['zt3'],
                            'e'   : normal(0, xt2**4, n)})


    table_data = pd.DataFrame({'Xt1' : data_new['zt1'],
                            'Xt2' : data_new['zt2'],
                            'Xt3' : data_new['zt3'],
                            'e'   : new_data['e']})
    table_data['Y'] = 4+0.4*table_data['Xt1']+1.5*table_data['Xt2']+3.6*table_data['Xt3']+table_data['e']
    table_data['Y_hat'] = 4+0.4*table_data['Xt1']+1.5*table_data['Xt2']+3.6*table_data['Xt3']
    table_data['e_hat']=table_data['Y']-table_data['Y_hat']
    table_data=table_data.reset_index(drop=True).reset_index()
    table_data["rep"]=rep
    table_data_all=pd.concat([table_data_all,table_data],axis=0,ignore_index=True,sort=False)
    # display(table_data)
    rep+=1
    if rep==100:
      break
table_data_all=table_data_all.set_index(["rep","index"])
table_data_all

Xt1       Xt2       Xt3  ...          Y      Y_hat     e_hat
rep  index                                ...                                
0.0  0.0   -1.338121 -0.651969  1.415615  ...   7.258908   7.583013 -0.324105
     1.0    0.987242  1.252671  0.537573  ...   7.923979   8.209167 -0.285187
     2.0    1.442430 -0.121335 -0.837442  ...   1.380182   1.380177  0.000005
     3.0   -1.872485 -0.494957 -1.684935  ...  -3.515547  -3.557195  0.041647
     4.0   -0.759111 -0.471591 -0.797517  ...   0.171996   0.117909  0.054087
...              ...       ...       ...  ...        ...        ...       ...
99.0 45.0   0.024150  0.480642 -1.116559  ...   0.815026   0.711010  0.104016
     46.0   0.354609 -0.478981 -0.725963  ...   0.732478   0.809906 -0.077428
     47.0   2.079922  1.063471  0.997728  ...   9.202639  10.018997 -0.816359
     48.0  -0.866984 -0.604879 -1.062870  ...  -1.135485  -1.080444 -0.055042
     49.0  -0.572212  0.779627  1.737142  ...  11.450927  11.194267  0.256660

[5000 rows x 7 columns]

In [15]:
table_data_all.mean()

Xt1      0.019700
Xt2     -0.003452
Xt3      0.001310
e        0.059372
Y        4.066790
Y_hat    4.007418
e_hat    0.059372
dtype: float64

In [16]:
table_data_all.var()

Xt1       1.011174
Xt2       0.986140
Xt3       0.987484
e        84.633591
Y        96.847691
Y_hat    15.216663
e_hat    84.633591
dtype: float64

In [17]:
# Compute pairwise correlation of columns, excluding NA/null values.
table_data_all.corr()

,Xt1,Xt2,Xt3,e,Y,Y_hat,e_hat
Xt1,1.000000,-0.009490,0.005638,-0.010315,0.031842,0.104660,-0.010315
Xt2,-0.009490,1.000000,0.003135,-0.067986,0.088559,0.383753,-0.067986
Xt3,0.005638,0.003135,1.000000,-0.016149,0.349125,0.918860,-0.016149
e,-0.010315,-0.067986,-0.016149,1.000000,0.918235,-0.041834,1.000000
Y,0.031842,0.088559,0.349125,0.918235,1.000000,0.357276,0.918235
Y_hat,0.104660,0.383753,0.918860,-0.041834,0.357276,1.000000,-0.041834
e_hat,-0.010315,-0.067986,-0.016149,1.000000,0.918235,-0.041834,1.000000
